# 25.2 — Reliability Value at Risk (VaR) via Monte Carlo (Companion Notebook)

This notebook estimates **reliability Value at Risk (VaR)** for a repairable-system portfolio by simulating:
- **failure counts** over a planning horizon using an NHPP (Crow–AMSAA) expectation (approximated with a Poisson count model),
- **downtime per failure**, and
- **economic loss** (downtime cost + repair cost).

Outputs:
- VaR at 95% and 99% for **Fleet** and **PMP**,
- loss distribution plots (300 dpi),
- CSV export for book / repo.


In [ ]:
# --- Imports (Google Colab / local Python) ---
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

np.random.seed(42)  # reproducibility


## 1) Inputs
### 1.1 Crow–AMSAA (NHPP power-law) parameters
From Chapter 24.4 (fitted from operational data):
- **Fleet:** β = 0.5472, η = 80.56
- **PMP:** β = 0.5196, η = 134.72

NHPP cumulative expected failures:
$$E[N(t)] = (t/\eta)^\beta$$
Expected failures over the horizon \([t_1, t_2]\):
$$\Delta E[N] = E[N(t_2)] - E[N(t_1)]$$


In [ ]:
# --- NHPP parameters (from your fitted results) ---
params = {
    "Fleet": {"beta": 0.5472, "eta": 80.56},
    "PMP":   {"beta": 0.5196, "eta": 134.72},
}

# --- Planning horizon definition ---
# Interpret t as "cumulative exposure" (e.g., operating hours).
# If you have absolute t_current and t_future from your fitted curve, set them here.
t1 = 8000.0      # baseline cumulative exposure (hours) - adjust
delta_t = 2000.0 # planning horizon length (hours) - adjust
t2 = t1 + delta_t

def EN(t, beta, eta):
    return (t/eta)**beta

def delta_EN(t1, t2, beta, eta):
    return EN(t2, beta, eta) - EN(t1, beta, eta)

for k,v in params.items():
    lam = delta_EN(t1, t2, v["beta"], v["eta"])
    print(f"{k}: ΔE[N] over [{t1:.0f}, {t2:.0f}] ≈ {lam:.3f} expected failures")


### 1.2 Economic loss model per failure
We convert reliability outcomes into loss:

$$\text{Loss} = (\text{Downtime hours} \times \text{Cost rate}) + \text{Repair cost}$$

Defaults below are generic—edit them to match your plant economics.


In [ ]:
# --- Economic assumptions (edit as needed) ---
# Downtime hours per failure: lognormal (right-skewed)
downtime_logn_mu = np.log(6.0)    # median ~ 6 hours
downtime_logn_sigma = 0.6         # variability

# Direct repair cost per failure: lognormal
repair_logn_mu = np.log(1200.0)   # median ~ 1,200 monetary units
repair_logn_sigma = 0.7

# Downtime cost rate (monetary units per hour)
downtime_cost_rate = 5000.0

# --- Simulation settings ---
n_sims = 20000
alpha_levels = [0.95, 0.99]


## 2) Monte Carlo Simulation
### 2.1 Failure count model
NHPP provides an expected count \(\Delta E[N]\). For planning, we model the number of failures in the horizon as:
$$K \sim \text{Poisson}(\lambda), \quad \lambda = \Delta E[N]$$
This yields a distribution of outcomes needed for VaR.


In [ ]:
def simulate_losses(beta, eta, n_sims=20000):
    lam = float(delta_EN(t1, t2, beta, eta))
    lam = max(lam, 0.0)

    # failures in horizon
    K = np.random.poisson(lam=lam, size=n_sims)

    total_loss = np.zeros(n_sims, dtype=float)
    for i, k in enumerate(K):
        if k <= 0:
            total_loss[i] = 0.0
            continue
        downtime = np.random.lognormal(mean=downtime_logn_mu, sigma=downtime_logn_sigma, size=k)
        repair = np.random.lognormal(mean=repair_logn_mu, sigma=repair_logn_sigma, size=k)
        total_loss[i] = downtime.sum() * downtime_cost_rate + repair.sum()

    return lam, K, total_loss

results = {}
for scope, p in params.items():
    lam, K, loss = simulate_losses(p["beta"], p["eta"], n_sims=n_sims)
    results[scope] = {"lambda": lam, "K": K, "loss": loss}

{scope: results[scope]["lambda"] for scope in results}


## 3) VaR Computation
VaR at confidence level \(\alpha\) is the \(\alpha\)-quantile of the loss distribution:
$$\mathrm{VaR}_\alpha = Q_\alpha(\text{Loss})$$


In [ ]:
def compute_var(loss, alpha):
    return float(np.quantile(loss, alpha))

rows = []
for scope in results:
    loss = results[scope]["loss"]
    row = {
        "Scope": scope,
        "NHPP_lambda_expected_failures": results[scope]["lambda"],
        "Mean_Loss": float(loss.mean()),
        "P(Loss=0)": float((loss == 0).mean()),
    }
    for a in alpha_levels:
        row[f"VaR_{int(a*100)}"] = compute_var(loss, a)
    rows.append(row)

summary_df = pd.DataFrame(rows).sort_values("Scope")
summary_df


## 4) Plot Loss Distributions (with VaR markers)
Figures are saved at **300 dpi** for inclusion in the book.


In [ ]:
import os
out_dir = "outputs_25_2"
os.makedirs(out_dir, exist_ok=True)

def plot_loss(scope, loss):
    var95 = compute_var(loss, 0.95)
    var99 = compute_var(loss, 0.99)
    xmax = np.quantile(loss, 0.995) if np.any(loss > 0) else 1.0

    plt.figure()
    plt.hist(loss[loss <= xmax], bins=60)
    plt.axvline(var95, linestyle="--", linewidth=2, label=f"VaR 95% = {var95:,.0f}")
    plt.axvline(var99, linestyle=":", linewidth=2, label=f"VaR 99% = {var99:,.0f}")
    plt.title(f"Loss Distribution — {scope} (Horizon: {delta_t:.0f} h)")
    plt.xlabel("Total loss in horizon")
    plt.ylabel("Frequency (simulations)")
    plt.legend()

    png = os.path.join(out_dir, f"VaR_LossDist_{scope}.png")
    plt.savefig(png, dpi=300, bbox_inches="tight")
    plt.show()
    return png

pngs = []
for scope in results:
    pngs.append(plot_loss(scope, results[scope]["loss"]))
pngs


## 5) Export Tables for the Companion Repo
Exports:
- Summary table (mean + VaR values)
- Loss samples (optional)


In [ ]:
summary_csv = os.path.join(out_dir, "Reliability_VaR_Summary_25_2.csv")
summary_df.to_csv(summary_csv, index=False)

loss_samples_csv = os.path.join(out_dir, "Reliability_VaR_LossSamples_25_2.csv")
pd.DataFrame({
    "Loss_Fleet": results["Fleet"]["loss"],
    "Loss_PMP": results["PMP"]["loss"],
}).to_csv(loss_samples_csv, index=False)

summary_csv, loss_samples_csv


## 6) Notes for the Book
- **λ (expected failures)** comes from the NHPP model over the chosen horizon.
- VaR depends strongly on **downtime cost rate** and **downtime distribution** assumptions.
- For report numbers, keep the horizon and cost assumptions consistent with your economics simulation.
